In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [23]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Medikament_Event CASCADE;
DROP TABLE IF EXISTS Event_Arzt CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;"""


In [24]:
cur.execute(drop_tables)
conn.commit()
print("Alle Tabellen wurden gelöscht.")

Alle Tabellen wurden gelöscht.


In [25]:
create_tables = """

-- =========================
-- CREATE TABLES
-- =========================

CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Name VARCHAR(100) NOT NULL,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    PLZ VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL,
    CHECK (PLZ ~ '^[0-9]{4}$')
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    CHECK (Kapazitaet > 0),
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Versicherungsnummer VARCHAR(30) UNIQUE,
    Elektronisches_Patientendossier_ID VARCHAR(50) UNIQUE,
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    PLZ VARCHAR(10),
    Stadt VARCHAR(50),
    Station_ID INT,
    CHECK (PLZ IS NULL OR PLZ ~ '^[0-9]{4}$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Telefonnummer VARCHAR(30),
    ist_extern BOOLEAN NOT NULL DEFAULT FALSE,
    Station_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflegekraft_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Telefonnummer VARCHAR(30),
    Station_ID INT NOT NULL,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Vorname VARCHAR(50),
    Nachname VARCHAR(50),
    Telefonnummer VARCHAR(30),
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$')
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Station_ID INT NOT NULL,
    CHECK (Status IN ('frei', 'belegt')),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Behandlung (
    Behandlung_ID INT PRIMARY KEY,
    Patient_ID INT NOT NULL,
    Datum DATE NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID)
);

CREATE TABLE Raum (
    Raum_ID INT PRIMARY KEY,
    Raum_Nr VARCHAR(20) NOT NULL,
    Art VARCHAR(50),
    Station_ID INT NOT NULL,
    CHECK (Art IS NULL OR Art IN ('OP', 'Untersuchung', 'Station', 'Sonstiges')),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Beschreibung VARCHAR(255),
    Datum DATE NOT NULL,
    Uhrzeit TIME NOT NULL,
    Raum_ID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    Abgeschlossen BOOLEAN NOT NULL DEFAULT FALSE,
    FOREIGN KEY (Raum_ID) REFERENCES Raum(Raum_ID),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlung_ID)
);

CREATE TABLE Event_Arzt (
    Event_ID INT NOT NULL,
    Arzt_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Name VARCHAR(100) NOT NULL UNIQUE,
    Wirkstoff VARCHAR(100)
);

CREATE TABLE Medikament_Event (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    Dosierung VARCHAR(50),
    Einheit VARCHAR(20),
    Zeitpunkt TIME,
    CHECK (Einheit IS NULL OR Einheit IN ('mg', 'ml', 'Tablette')),
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);

"""


cur.execute(create_tables)
conn.commit()
print("Alle Tabellen wurden neu erstellt.")



Alle Tabellen wurden neu erstellt.


In [ ]:
from faker import Faker
fake = Faker("de_CH")


In [ ]:
from faker import Faker
fake = Faker("de_CH")

# =========================
# Krankenhaus
# =========================
insert_krankenhaus = """
INSERT INTO Krankenhaus (KH_ID, Name, Strasse, Hausnummer, PLZ, Stadt)
VALUES (%s, %s, %s, %s, %s, %s)
"""
for kh_id in range(1, 4):
    cur.execute(insert_krankenhaus, (
        kh_id,
        f"Krankenhaus {kh_id}",
        fake.street_name(),
        fake.building_number(),
        fake.postcode(),
        fake.city()
    ))
conn.commit()

# =========================
# Station
# =========================
insert_station = """
INSERT INTO Station (Station_ID, Name, Kapazitaet, KH_ID)
VALUES (%s, %s, %s, %s)
"""
station_id = 1
for kh_id in range(1, 4):
    for _ in range(3):
        cur.execute(insert_station, (
            station_id,
            fake.word().capitalize(),
            fake.random_int(10, 40),
            kh_id
        ))
        station_id += 1
conn.commit()

# =========================
# Patient
# =========================
insert_patient = """
INSERT INTO Patient (
    Patient_ID, Vorname, Nachname, Geburtsdatum,
    Versicherungsnummer, Elektronisches_Patientendossier_ID,
    Strasse, Hausnummer, PLZ, Stadt, Station_ID
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""
for pid in range(1, 51):
    cur.execute(insert_patient, (
        pid,
        fake.first_name(),
        fake.last_name(),
        fake.date_of_birth(minimum_age=0, maximum_age=90),
        fake.bothify("CH########"),
        fake.uuid4(),
        fake.street_name(),
        fake.building_number(),
        fake.postcode(),
        fake.city(),
        fake.random_int(1, station_id - 1)
    ))
conn.commit()

# =========================
# Arzt
# =========================
insert_arzt = """
INSERT INTO Arzt (Arzt_ID, Vorname, Nachname, Telefonnummer, ist_extern, Station_ID)
VALUES (%s, %s, %s, %s, %s, %s)
"""
arzt_id = 1
for _ in range(15):
    cur.execute(insert_arzt, (
        arzt_id,
        fake.first_name(),
        fake.last_name(),
        fake.phone_number(),
        fake.boolean(chance_of_getting_true=20),
        fake.random_int(1, station_id - 1)
    ))
    arzt_id += 1
conn.commit()

# =========================
# Pflegekraft
# =========================
insert_pflegekraft = """
INSERT INTO Pflegekraft (Pflegekraft_ID, Vorname, Nachname, Telefonnummer, Station_ID)
VALUES (%s, %s, %s, %s, %s)
"""
pflege_id = 1
for _ in range(20):
    cur.execute(insert_pflegekraft, (
        pflege_id,
        fake.first_name(),
        fake.last_name(),
        fake.phone_number(),
        fake.random_int(1, station_id - 1)
    ))
    pflege_id += 1
conn.commit()

# =========================
# Reserve
# =========================
insert_reserve = """
INSERT INTO Reserve (Reserve_ID, Vorname, Nachname, Telefonnummer)
VALUES (%s, %s, %s, %s)
"""
for rid in range(1, 6):
    cur.execute(insert_reserve, (
        rid,
        fake.first_name(),
        fake.last_name(),
        fake.phone_number()
    ))
conn.commit()

# =========================
# Bett
# =========================
insert_bett = """
INSERT INTO Bett (Bett_ID, Status, Station_ID)
VALUES (%s, %s, %s)
"""
bett_id = 1
for sid in range(1, station_id):
    for _ in range(5):
        cur.execute(insert_bett, (
            bett_id,
            fake.random_element(["frei", "belegt", "gesperrt"]),
            sid
        ))
        bett_id += 1
conn.commit()

# =========================
# Behandlung
# =========================
insert_behandlung = """
INSERT INTO Behandlung (Behandlung_ID, Patient_ID, Datum)
VALUES (%s, %s, %s)
"""
for bid in range(1, 51):
    cur.execute(insert_behandlung, (
        bid,
        bid,
        fake.date_between(start_date="-1y", end_date="today")
    ))
conn.commit()

# =========================
# Raum
# =========================
insert_raum = """
INSERT INTO Raum (Raum_ID, Raum_Nr, Art, Station_ID)
VALUES (%s, %s, %s, %s)
"""
raum_id = 1
for sid in range(1, station_id):
    for _ in range(3):
        cur.execute(insert_raum, (
            raum_id,
            fake.bothify("R-###"),
            fake.random_element(["OP", "Untersuchung", "Behandlungsraum"]),
            sid
        ))
        raum_id += 1
conn.commit()

# =========================
# Event
# =========================
insert_event = """
INSERT INTO Event (
    Event_ID, Beschreibung, Datum, Uhrzeit,
    Raum_ID, Behandlung_ID, Abgeschlossen
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
for eid in range(1, 51):
    cur.execute(insert_event, (
        eid,
        fake.sentence(),
        fake.date_between(start_date="-6m", end_date="today"),
        fake.time(),
        fake.random_int(1, raum_id - 1),
        eid,
        fake.boolean()
    ))
conn.commit()

# =========================
# Event_Arzt
# =========================
insert_event_arzt = """
INSERT INTO Event_Arzt (Event_ID, Arzt_ID)
VALUES (%s, %s)
"""
for eid in range(1, 51):
    cur.execute(insert_event_arzt, (
        eid,
        fake.random_int(1, arzt_id - 1)
    ))
conn.commit()

# =========================
# Medikament
# =========================
insert_medikament = """
INSERT INTO Medikament (Medikament_ID, Name, Wirkstoff)
VALUES (%s, %s, %s)
"""
for mid in range(1, 11):
    cur.execute(insert_medikament, (
        mid,
        fake.word().capitalize(),
        fake.word()
    ))
conn.commit()

# =========================
# Medikament_Event
# =========================
insert_med_event = """
INSERT INTO Medikament_Event (
    Event_ID, Medikament_ID, Dosierung, Einheit, Zeitpunkt
)
VALUES (%s, %s, %s, %s, %s)
"""
for eid in range(1, 51):
    cur.execute(insert_med_event, (
        eid,
        fake.random_int(1, 10),
        fake.random_int(1, 3),
        "mg",
        fake.time()
    ))
conn.commit()


In [20]:
conn.rollback()
